**WebScraping Com Python**


In [1]:
#IMPORTANDO AS BIBLIOTECAS
from urllib.request import Request, urlopen
import bs4
import pandas as pd
import time
import pyodbc

In [2]:
#INFORMANDO O AGENTE
agente = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0'
headers = {'User-Agent': agente}
pd.options.display.max_colwidth = 100

In [3]:
#WEBSCRAPPING COM PANDAS
# FUNÇÃO PARA CHAMAR A PÁGINA WEB
def ConsultaWeb(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass
    
# FUNÇÃO PARA PEGAR A PÁGINA ENCONTRADA E TRANSFORMAR EM UM OBJETO UTILIZÁVEL    
def captura_html_pagina(url):
    html = ConsultaWeb(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

# FUNÇÃO PARA CAPTURAR AS INFORMAÇÕES DA "TABELINHA CABEÇALHO" DA PÁGINA
def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x] = y
    return dic

# FUNÇÃO PARA CHECAR SE TEM ALGUM ERRO NA PÁGINA
def ChecaErro(url):
    html = captura_html_pagina(url)
    Erro = html.find(class_="col-10")
    if(Erro==None):
        return True
    else:
        return False
    
# FUNÇÃO PARA TRAZER O CABEÇALHO DA PROPOSIÇÃO E TRANSFORMAR EM DICIONÁRIO    
def ProposicaoCabecalho(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+
    str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [4]:
# FUNÇÃO PARA FAZER SELEÇÃO DE DADOS
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial',
                          UID='sa',
                          PWD='SeilaW1F1'
                         )

    out = pd.read_sql_query(query,conn)
    return out

# FUNÇÃO PARA FAZER A INSERÇÃO DOS DADOS NA TABELA PROPOSICOES    
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial',
                          UID='sa',
                          PWD='SeilaW1F1'
                         )

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']

        )

    conn.commit()
    cursor.close()

# FUNÇÃO DE LIMPEZA DE DADOS DA TABELA    
def SQLTruncate(NomeTabela):

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial',
                          UID='sa',
                          PWD='SeilaW1F1'

                         )

    cursor = conn.cursor()

    cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

def InsereProximaProposicao(ano):

    # Busca última proposição cadastrada
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = ProposicaoCabecalho(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)
    
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 20, segundos_espera = 0):

    # erros
    erros = 0
    qty = 1 #preposição
    
    # variaveis para loop
    i = 1
    lista = []
    
    while erros <= erros_admissiveis and qty <= quantidade:

        try:
            qty += 1
            url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(qty)+'&aaProposicao='+str(ano)
            if(ChecaErro(url)):
                InsereProximaProposicao(ano)
            else:
                erros+=1
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [5]:
ano_inicial = 1996
ano_final = 2023

for i in list(range(ano_inicial, ano_final+1)):
    print('INICIANDO GRAVAÇÃO DO DADOS DO ANO: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 999999999999)
    except:
        pass
    
print('INSERÇÃO FINALIZADA COM SUCESSO:')

INICIANDO GRAVAÇÃO DO DADOS DO ANO:  1996
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  1997
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  1998
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  1999
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2000
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2001
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2002
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2003
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2004
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2005
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2006
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2007
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2008
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2009
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2010
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2011
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2012
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2013
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2014
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2015
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2016
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2017
INICIANDO GRAVAÇÃO DO DADOS DO ANO:  2018
INICIANDO GRAVAÇÃO DO DADOS DO ANO

In [ ]:
# VISUALIZANDO A TABELA
SQLSelect('''
    select 
        *
    from Proposicoes
''')

In [ ]:
# LIMPANDO DADOS DA TABELA
SQLTruncate('Proposicoes')

In [ ]:
#SQLSelect('''
#    select 
#        *
#    from Proposicoes
#''')

In [ ]:
preposicao = 365
ano = 2022
url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(preposicao)+'&aaProposicao='+str(ano)
html = captura_html_pagina(url)
x = ChecaErro(url)
x